In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
from zipfile import ZipFile
import urllib
from datetime import datetime
import string
import pygsheets
import requests
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import Bio.pairwise2 as pairwise2
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.viennarna as viennarna
import paulssonlab.cloning.thermodynamics as thermodynamics

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Backbone insulation

In [ ]:
import Bio.SeqUtils.MeltingTemp as MeltingTemp
import primer3

In [ ]:
olib = reg[("oLIB", "oligos")]
plib_maps = reg[("pLIB", "maps")]
primers = {f"oLIB{num}": olib[f"oLIB{num}"]["Sequence"] for num in range(179, 183)}

In [ ]:
template = plib_maps["pLIB122"]

In [ ]:
seq = "cctttaattaaggagttttg"

In [ ]:
# seq = "AACACACAGACGAGACGCGGAA"
MeltingTemp.Tm_NN(
    seq, dnac1=500, nn_table=MeltingTemp.DNA_NN3, saltcorr=7, Na=150
)  # + 9

In [ ]:
primers["oLIB180"]

In [ ]:
sequence.find_primer_binding_site?

In [ ]:
sequence.find_primer_binding_site(
    template.seq_lower(), primers["oLIB180"].lower(), reverse_complement=None
)

In [ ]:
primer_conc = 500

primer3.calcTm(
    seq,
    dna_conc=(primer_conc / 6) * 7,  # primer is assumed 6x template
    mv_conc=200,  # (60+20),
    dv_conc=2,
    tm_method="santalucia",
    salt_corrections_method="owczarzy",
)

In [ ]:
primer_conc = 500

primer3.calcTm(
    seq,
    dna_conc=(primer_conc / 6) * 7,  # primer is assumed 6x template
    mv_conc=200,  # (60+20),
    dv_conc=0,
    tm_method="santalucia",
    salt_corrections_method="owczarzy",
)

In [ ]:
orig_upstream = (primers["oLIB179"], primers["oLIB180"])

In [ ]:
[viennarna.dna_secondary_structure(s) for s in orig_upstream]

In [ ]:
viennarna.dna_heterodimer(*orig_upstream)

In [ ]:
orig_downstream = (primers["oLIB181"], primers["oLIB182"])

In [ ]:
[viennarna.dna_secondary_structure(s) for s in orig_downstream]

In [ ]:
viennarna.dna_heterodimer(*orig_downstream)

In [ ]:
upstream = ("CCTTTAATTAAGGAGTTTTGCAGGTGC", "CATCAAATAAAACGAAAGGCTCAGTCG")

In [ ]:
[viennarna.dna_secondary_structure(s) for s in upstream]

In [ ]:
viennarna.dna_heterodimer(*upstream)

In [ ]:
s = "cctttaattaaggagttttgcaggtgcctt"[:]
[viennarna.dna_secondary_structure(s[:-i]) for i in reversed(range(1, 20))]

In [ ]:
[s[:-i] for i in reversed(range(1, 20))]

In [ ]:
downstream = ("caagactagagcgaagtcttcgaag", "gactcctgttgatagatccagtaatgac")

In [ ]:
viennarna.dna_heterodimer(*downstream)

# Backbone primers with UNSes for FISH barcodes

In [ ]:
# backbone primers with UNS9 instead of UNS1 (reverse) and UNS10 (forward)

In [ ]:
str.replace?

In [ ]:
lib_parts = reg[("LIB", "parts")]
plib_maps = reg[("pLIB", "maps")]
olib_oligos = reg[("oLIB", "oligos")]

In [ ]:
uns1 = lib_parts["UNS1"]["Sequence"]
uns9 = lib_parts["UNS9"]["Sequence"]
uns10 = lib_parts["UNS10"]["Sequence"]

In [ ]:
target = plib_maps["pLIB122"]

In [ ]:
old_primer = olib_oligos["oLIB177"]["Sequence"]
new_primer = workflow.replace_primer_tail(target, old_primer)
print(old_primer)
print(new_primer)

In [ ]:
old_primer = olib_oligos["oLIB177"]["Sequence"]
new_primer = workflow.replace_primer_tail(target, old_primer, "AAAA")
print(old_primer)
print(new_primer)

# FISH barcode primers with UNSes

In [ ]:
upstream = "tctcctacataatacgactcactatagggacgaacgttagcagcactat"
downstream = "aaccccttggggcctctaaacgggtcttgaggggttttttgggaatgaga"

In [ ]:
# "caaaaaacccctcaagac" + uns9

In [ ]:
%%time
enumerate_primers(
    sequence.reverse_complement(downstream).lower(),
    tail=uns9,
    min_mfe=-20,
    monotonic_mfe=False,
    anchor_3prime=False,
)

In [ ]:
enumerate_primer_pairs(upstream, downstream)

## Primer3

In [ ]:
import primer3plus

In [ ]:
design = primer3plus.Design()
design.settings.template(str(sequence.get_seq(target)))
# design.settings.as_cloning_task()
design.settings.primer_num_return(1)
results, explain = design.run()

In [ ]:
explain

In [ ]:
results

In [ ]:
enzyme.re_search

In [ ]:
test_seq = "gcttcacgtctcggtctcaaggtctttaaaaagaaacctccgcattgcggaggtttcgccttttgatactctgtctgaagtaattcttgccgcagtgaaaaatggcgcccatcggcgccatttttttatgcttccattagaaagcaaaaagcctgctagaaagcaggcttttttgaatttggctcctctgacgctttgagaccggagacgtgctaa"

In [ ]:
enzyme._re_search(test_seq, Restriction.BsaI)

In [ ]:
test_seq[12:197]

In [ ]:
# find BsaI sites
# hhh

In [ ]:
import primer3

In [ ]:
primer3.designPrimers(
    {
        "SEQUENCE_TEMPLATE": str(sequence.get_seq(target)),
        "SEQUENCE_INCLUDED_REGION": [36, 342],
    },
    {
        #         'PRIMER_PICK_INTERNAL_OLIGO': 1,
        #         'PRIMER_INTERNAL_MAX_SELF_END': 8,
        "PRIMER_OPT_SIZE": 20,
        "PRIMER_MIN_SIZE": 18,
        "PRIMER_MAX_SIZE": 25,
        "PRIMER_OPT_TM": 60.0,
        "PRIMER_MIN_TM": 57.0,
        "PRIMER_MAX_TM": 63.0,
        "PRIMER_MIN_GC": 20.0,
        "PRIMER_MAX_GC": 80.0,
        #         'PRIMER_MAX_POLY_X': 100,
        #         'PRIMER_INTERNAL_MAX_POLY_X': 100,
        "PRIMER_SALT_MONOVALENT": 50.0,
        "PRIMER_DNA_CONC": 50.0,
        #         'PRIMER_MAX_NS_ACCEPTED': 0,
        #         'PRIMER_MAX_SELF_ANY': 12,
        #         'PRIMER_MAX_SELF_END': 8,
        #         'PRIMER_PAIR_MAX_COMPL_ANY': 12,
        #         'PRIMER_PAIR_MAX_COMPL_END': 8,
        #         'PRIMER_PRODUCT_SIZE_RANGE': [[75,100],[100,125],[125,150],
        #                                       [150,175],[175,200],[200,225]],
    },
)

# Old

In [ ]:
from cytoolz import dissoc

In [ ]:
olt = reg[("oLT", "oligos")]

In [ ]:
old_row = olt["oLT74"]
seq = old_row["Sequence"]

In [ ]:
len(seq)

In [ ]:
homology_length = 20
break_idx = 111
idxs = (slice(None, break_idx + homology_length), slice(break_idx, None))
for idx in idxs:
    print(seq[idx])

In [ ]:
date = datetime.now().strftime("%-m/%-d/%Y")

rows = []

for i, idx in enumerate(idxs):
    row = {
        **dissoc(old_row, "ID"),
        "Sequence": seq[idx].upper(),
        "Name": "{}_v2_split{}".format(old_row["Name"], i + 1),
        "Description": "Split section {}-of-{} of oLT74 (20bp overlap).\n{}".format(
            i + 1, len(idxs), old_row["Description"]
        ),
        "Date": date,
        "Order date": date,
    }
    rows.append(row)

In [ ]:
for row in rows:
    olt[olt.next_id()] = row

In [ ]:
olt.save()

In [ ]:
rows